In [1]:
import pandas as pd
from scipy import *
from scipy.sparse import *
import os

In [2]:
%load_ext Cython

In [3]:
import numpy as np

In [4]:
from tqdm.auto import tqdm

In [5]:
sim_name = load_npz(os.path.join('../dataset/validation/similarities/jaccard_uncleaned_name_300k_validation_2ngrams.npz'))    
sim_email = load_npz(os.path.join('../dataset/validation/similarities/jaccard_uncleaned_email_300k_validation_2ngrams.npz'))    
sim_phone = load_npz(os.path.join('../dataset/validation/similarities/jaccard_uncleaned_phone_300k_validation_2ngrams.npz'))    
sim_address = load_npz(os.path.join('../dataset/validation/similarities/jaccard_uncleaned_address_300k_validation_2ngrams.npz'))    

In [6]:
hybrid = sim_name + 0.2 * sim_email + 0.2 * sim_phone + 0.2 * sim_address

In [7]:
indices = hybrid[1].nonzero()[1][hybrid[1].data.argsort()[::-1]][:50]

In [8]:
indices

array([     1, 167911,  31141,  31142,  49483,  52528,  52530,  52529,
        87312,  87314,  87311,  87313, 171753, 171751, 171752, 131766,
        28454, 131767,  22580,  22581,  22582,  22584,  22585,  22588,
        22583, 148113, 148114, 148115, 258124,  48989, 112878, 112879,
       112880, 112881, 229279, 229278,  28458,  28457,  97455,  34445,
       188794, 109067,  98554, 440753, 425852, 211926, 111801,  27442,
        88923,  27443], dtype=int32)

In [9]:
name_row = sim_name[1, indices]

In [10]:
name_row.toarray()[0]

array([1.        , 0.5208333 , 0.50980395, 0.50980395, 0.48      ,
       0.4509804 , 0.4509804 , 0.4509804 , 0.44680852, 0.44680852,
       0.44680852, 0.44680852, 0.44067797, 0.44067797, 0.44067797,
       0.4375    , 0.4375    , 0.4375    , 0.43396226, 0.43396226,
       0.43396226, 0.43396226, 0.43396226, 0.43396226, 0.43396226,
       0.43137255, 0.43137255, 0.43137255, 0.42857143, 0.42307693,
       0.42307693, 0.42307693, 0.42307693, 0.42307693, 0.42      ,
       0.42      , 0.42      , 0.42      , 0.41666666, 0.41666666,
       0.41666666, 0.41666666, 0.41509435, 0.41509435, 0.41509435,
       0.41509435, 0.4117647 , 0.4074074 , 0.4074074 , 0.4074074 ],
      dtype=float32)

In [7]:
df_test = pd.read_csv("../dataset/validation/test.csv", escapechar="\\")

In [8]:
df_test = df_test.sort_values(by=['record_id']).reset_index(drop=True)

In [9]:
df_train = pd.read_csv("../dataset/validation/train.csv", escapechar="\\")
df_train = df_train.sort_values(by=['record_id']).reset_index(drop=True)

In [12]:
linid_ = []
linid_idx = []
linid_score = []
linid_name_cosine = []
linid_email_cosine = []
linid_phone_cosine = []
linid_address_cosine = []
linid_record_id = []
k=50

tr = df_train[['record_id', 'linked_id']]
for x in tqdm(range(df_test.shape[0])):
    #df = df_train.loc[hybrid[x].nonzero()[1][hybrid[x].data.argsort()[::-1]],:][:k]
    indices = hybrid[x].nonzero()[1][hybrid[x].data.argsort()[::-1]][:k]
    #df = tr.loc[indices, :][:k]
    #linid_.append(df['linked_id'].values)
    linid_idx.append(indices)    # TODO CAMBIATO QUI
    #linid_record_id.append(df.record_id.values)
    #linid_score.append(np.sort(hybrid[x].data)[::-1][:k]) # Questo ha senso perché tanto gli indices sono sortati in base allo scores di hybrid
    #linid_name_cosine.append([sim_name[x, t] for t in indices])
    #linid_email_cosine.append([sim_email[x, t] for t in indices])
    #linid_phone_cosine.append([sim_phone[x, t] for t in indices])
    #linid_address_cosine.append([sim_address[x, t] for t in indices])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [323]:
linid_name_cosine[2]

[1.0,
 0.53846157,
 0.53846157,
 0.375,
 0.42857143,
 0.42857143,
 0.42857143,
 0.42857143,
 0.4,
 0.3125,
 0.375,
 0.375,
 0.375,
 0.375,
 0.375,
 0.375,
 0.35714287,
 0.35714287,
 0.35714287,
 0.3529412,
 0.3529412,
 0.3529412,
 0.3529412,
 0.3529412,
 0.3529412,
 0.3529412,
 0.3529412,
 0.2631579,
 0.2777778,
 0.2777778,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.33333334,
 0.31578946,
 0.31578946,
 0.31578946]

## Not Correct Cython : wrong order of returned values

In [319]:
%%cython
import numpy as np
from tqdm import tqdm_notebook as tqdm

def extract_values(list idxs, int k, float[:] data, int[:] indices, int[:] indptr):
    
    cdef Py_ssize_t x_idxs = len(idxs)
    cdef int x
    cdef int t
    cdef int indptr_start
    cdef int indptr_end
    cdef float[:] row_data
    cdef long[:] available_idxs
    
    res = np.zeros((x_idxs, k), dtype=float)
    cdef double[:, :] res_view = res
    
    for x in tqdm(range(x_idxs)):
        indptr_start = indptr[x]
        indptr_end = indptr[x+1]
        #available_idxs = list(set(idxs[x]).intersection(set(indices[indptr_start: indptr_end])))
        available_idxs = np.where(np.isin(indices[indptr_start : indptr_end], idxs[x]))[0]
        row_data = data[indptr_start:indptr_end]
        for t in range(len(available_idxs)):
            res_view[x, t] = row_data[available_idxs[t]] 
        
    return res

In [268]:
a = extract_values(linid_idx, 50, sim_name.data, sim_name.indices, sim_name.indptr)

KeyboardInterrupt: 

## Correct cython code!!

In [13]:
%%cython
import numpy as np
from tqdm import tqdm_notebook as tqdm

cdef list get_inorder_sliced(int[:] tmp_idxs, int[:] sliced, long[:] available_idxs):
    cdef dict idxs_dict
    cdef list tuple_sliced
    
    idxs_dict = {k:v for v,k in enumerate(tmp_idxs)}
    tuple_sliced = [( idxs_dict[sliced[i]] , i)   for i in available_idxs]
    tuple_sliced.sort(key=lambda tup: tup[0])
    tuple_sliced = [x[1] for x in tuple_sliced]
    return tuple_sliced
    

cpdef extract_values_2(list idxs, int k, float[:] data, int[:] indices, int[:] indptr):
    
    cdef Py_ssize_t x_idxs = len(idxs)
    cdef int x
    cdef int t
    cdef int indptr_start
    cdef int indptr_end
    cdef float[:] row_data
    
    cdef long[:] available_idxs
    cdef list ordered_available
            
    res = np.zeros((x_idxs, k), dtype=float)
    cdef double[:, :] res_view = res
    
    for x in tqdm(range(x_idxs)):
        indptr_start = indptr[x]
        indptr_end = indptr[x+1]
        
    
        available_idxs = np.where(np.isin(indices[indptr_start : indptr_end], idxs[x]))[0]
    
        ordered_available = get_inorder_sliced(idxs[x], indices[indptr_start : indptr_end], available_idxs)
    
        row_data = data[indptr_start:indptr_end]
        for t in range(len(ordered_available)):
            res_view[x, t] = row_data[ordered_available[t]] 
    
    return res

In [14]:
a = extract_values_2(linid_idx, 50, sim_name.data, sim_name.indices, sim_name.indptr)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [324]:
a[2]

array([1.        , 0.53846157, 0.53846157, 0.375     , 0.42857143,
       0.42857143, 0.42857143, 0.42857143, 0.40000001, 0.3125    ,
       0.375     , 0.375     , 0.375     , 0.375     , 0.375     ,
       0.375     , 0.35714287, 0.35714287, 0.35714287, 0.35294119,
       0.35294119, 0.35294119, 0.35294119, 0.35294119, 0.35294119,
       0.35294119, 0.35294119, 0.2631579 , 0.27777779, 0.27777779,
       0.33333334, 0.33333334, 0.33333334, 0.33333334, 0.33333334,
       0.33333334, 0.33333334, 0.33333334, 0.33333334, 0.33333334,
       0.33333334, 0.33333334, 0.33333334, 0.33333334, 0.33333334,
       0.33333334, 0.33333334, 0.31578946, 0.31578946, 0.31578946])

In [ ]:
linid_ = []
linid_idx = []
linid_score = []
linid_name_cosine = []
linid_email_cosine = []
linid_phone_cosine = []
linid_address_cosine = []
linid_record_id = []
k=50

tr = df_train[['record_id', 'linked_id']]
for x in tqdm(range(df_test.shape[0])):
    #df = df_train.loc[hybrid[x].nonzero()[1][hybrid[x].data.argsort()[::-1]],:][:k]
    indices = hybrid[x].nonzero()[1][hybrid[x].data.argsort()[::-1]][:k]
    df = tr.loc[indices, :][:k]
    linid_.append(df['linked_id'].values)
    linid_idx.append(indices)    # TODO CAMBIATO QUI
    linid_record_id.append(df.record_id.values)
    linid_score.append(np.sort(hybrid[x].data)[::-1][:k]) # Questo ha senso perché tanto gli indices sono sortati in base allo scores di hybrid
    linid_name_cosine.append([sim_name[x, t] for t in indices])
    #linid_email_cosine.append([sim_email[x, t] for t in indices])
    #linid_phone_cosine.append([sim_phone[x, t] for t in indices])
    #linid_address_cosine.append([sim_address[x, t] for t in indices])
    

In [325]:
extract_values_2(linid_idx, 50, sim_name.data, sim_name.indices, sim_name.indptr)
extract_values_2(linid_idx, 50, sim_email.data, sim_email.indices, sim_email.indptr)
extract_values_2(linid_idx, 50, sim_phone.data, sim_phone.indices, sim_phone.indptr)
extract_values_2(linid_idx, 50, sim_address.data, sim_address.indices, sim_address.indptr)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])